## Endpoints DCAT

Recommandations pour le POC Qualité : 
- Pas d'impératif de travailler sur du temps réel, possible de s'appuyer un dump des graphes ou de la base PostgreSQL

Liens utiles : 
- [Endpoint DCAT POC CKAN](https://preprod.data.developpement-durable.gouv.fr/dcat/catalog/jsonld?page=1), exemple page 1.
- [Endpoint DCAT data.gouv](https://www.data.gouv.fr/catalog.jsonld?page=1), exemple page 1.
- [Exemple d'endpoint DCAT GeoNetwork via rdf.search, datARA](https://catalogue.datara.gouv.fr/geonetwork/srv/fre/rdf.search?).
- [Exemple d'endpoint DCAT GeoNetwork via CSW, service bêta de la Géoplateforme](https://data.geopf.fr/csw?SERVICE=CSW&VERSION=2.0.2&REQUEST=GetRecords&ElementSetName=full&ResultType=results&MaxRecords=10&OutputFormat=application/xml&OutputSchema=http%3A%2F%2Fwww.w3.org%2Fns%2Fdcat%23&NAMESPACE=xmlns%28dcat%3Dhttp%3A%2F%2Fwww.w3.org%2Fns%2Fdcat%23%29&TypeNames=dcat).
- [Exemple d'endpoint DCAT GeoNetwork via CSW, Geo-IDE](http://catalogue.geo-ide.developpement-durable.gouv.fr/catalogue/srv/eng/csw-moissonnable?service=CSW&REQUEST=GetRecordById&version=2.0.2&namespace=xmlns:csw=http://www.opengis.net/cat/csw&outputFormat=application/xml&outputSchema=http://www.w3.org/ns/dcat%23&ElementSetName=full&Id=fr-120066022-jdd-a307d028-d9d2-4605-a1e5-8d31bc573bef).
- [Documentation API data.gouv](https://doc.data.gouv.fr/api/reference/#/), voir en particulier */site/catalog.{format}*
- [Documentation data.gouv](https://doc.data.gouv.fr/api/telecharger-un-catalogue-de-donnees/) spécifiquement sur le téléchargement et la consulation du catalogue en RDF.
- [Documention GeoNetwork exposition DCAT rdf.search](https://geonetwork-opensource.org/manuals/4.0.x/en/api/rdf-dcat.html) endpoint DCAT.
- [Librairie Python RDFLib](https://rdflib.readthedocs.io/en/stable/intro_to_graphs.html) : navigation dans un graph RDF.

Tests :
- POC CKAN : comparer le temps de requête d'un graphe entre différentes pages.
- Extraire des métadonnées et les convertir en DataFrame.
- Comparer les expositions DCAT data.gouv, GeoNetworks, POC CKAN

### DCAT POC CKAN

#### Temps de réponse de l'API

In [ ]:
import rdflib
import time
import pandas as pd

In [ ]:
CKAN_INTEGRATION = "https://integration.data.e2.rie.gouv.fr/dcat/catalog/jsonld?page=1"
CKAN_PREPPROD = "https://preprod.data.developpement-durable.gouv.fr/dcat/catalog/jsonld?page=1"

In [ ]:
# Evaluation du temps de réponse de l'API (~30-40s par page)
pages = [1, 50, 100, 200]

for page in pages:
    try:
        start_time = time.time()
        rdflib.Graph().parse(f"https://preprod.data.developpement-durable.gouv.fr/dcat/catalog/jsonld?page={page}")
        print(f"Page {page}:", time.time()-start_time)
    except Exception as exception:
        print(f"Page {page}: ", exception)

In [ ]:
### Dump .json des expositions DCAT de l'environnement integration.
from pathlib import Path
from datetime import date
import glob

for page in range(1, 254):
    file_name = f"tmp/dumps/integration_page_{page}_" + str(date.today()) + ".json"
    if file_name not in glob.glob("tmp/dumps/*.json"):
        print(f"Processing page {page}")
        try:
            graph_ckan = rdflib.Graph().parse(f"https://integration.data.e2.rie.gouv.fr/dcat/catalog/jsonld?page={page}")
            file_name = f"integration_page_{page}_" + str(date.today())
            graph_ckan.serialize(destination = Path(file_name), format="json-ld", auto_compact=True)
        except Exception as exception:
            print(f"Fail to load graph for page {page}")

#### Récupération de l'URI du datasets, du titre, de la description, de la date de modification et de la licence

In [6]:
import glob
from utils.dcat_reader_ckan import DCATReaderCKAN
import pandas as pd

data = []
iteration = 0
for graph in glob.glob("tmp/dumps/*.json"):
    page_data = DCATReaderCKAN(graph).get_data()
    print("Fichier: ", graph, "- Nombre de datasets : ", len(page_data))
    data.append(
        page_data
    )
    iteration += 1

print("Nombre de pages traitées : ", iteration)
data = pd.concat(data)

Fichier:  tmp/dumps/integration_page_187_2023-10-14.json - Nombre de datasets :  55
Fichier:  tmp/dumps/integration_page_124_2023-10-14.json - Nombre de datasets :  98
Fichier:  tmp/dumps/integration_page_198_2023-10-14.json - Nombre de datasets :  5
Fichier:  tmp/dumps/integration_page_55_2023-10-14.json - Nombre de datasets :  100
Fichier:  tmp/dumps/integration_page_227_2023-10-14.json - Nombre de datasets :  0
Fichier:  tmp/dumps/integration_page_50_2023-10-14.json - Nombre de datasets :  100
Fichier:  tmp/dumps/integration_page_71_2023-10-14.json - Nombre de datasets :  100
Fichier:  tmp/dumps/integration_page_49_2023-10-14.json - Nombre de datasets :  100
Fichier:  tmp/dumps/integration_page_98_2023-10-14.json - Nombre de datasets :  100
Fichier:  tmp/dumps/integration_page_127_2023-10-14.json - Nombre de datasets :  56
Fichier:  tmp/dumps/integration_page_128_2023-10-14.json - Nombre de datasets :  56
Fichier:  tmp/dumps/integration_page_82_2023-10-14.json - Nombre de datasets :

In [2]:
data.describe(include="all")

,dataset
count,12877
unique,12877
top,http://catalogue.geo-ide.developpement-durable...
freq,1


### data.gouv
#### Récupération de l'URI du datasets, du titre, de la description, de la date de modification et de la licence

In [ ]:
graph = rdflib.Graph().parse("https://www.data.gouv.fr/catalog.jsonld?page=1")

In [ ]:
data = []
for subject, predicate, object in graph.triples((None,rdflib.term.URIRef("http://www.w3.org/1999/02/22-rdf-syntax-ns#type"),rdflib.term.URIRef("http://www.w3.org/ns/dcat#Dataset"))):
    data.append([subject,
                graph.value(subject=subject, predicate=rdflib.term.URIRef("http://purl.org/dc/terms/title")),
                graph.value(subject=subject, predicate=rdflib.term.URIRef("http://purl.org/dc/terms/description")),
                graph.value(subject=subject, predicate=rdflib.term.URIRef("http://purl.org/dc/terms/modified")), 
                graph.value(subject=subject, predicate=rdflib.term.URIRef("http://purl.org/dc/terms/license")),
                graph.value(subject=subject, predicate=rdflib.term.URIRef("http://www.w3.org/ns/dcat#startDate")),
                graph.value(subject=subject, predicate=rdflib.term.URIRef("http://www.w3.org/ns/dcat#endDate")),
                ])
                    
datasets_data_gouv = pd.DataFrame(data, columns= ["Object",
                                        "Title",
                                        "Description",
                                        "Modified",
                                        "Licence",
                                        "Start Date",
                                        "End Date"])